In [4]:
import polars as pl

In [11]:
post_file = 'data/monerosupport_submissions'
comments_file = 'data/monerosupport_comments'

In [18]:
p_df = pl.scan_ndjson(
    post_file
).select(
    pl.col(['id', 'url', 'num_comments', 'subreddit', 'title', 'selftext', 'score'])
).filter(
    (pl.col("score") > 0) &
    (pl.col('selftext') != '[deleted]') &
    (pl.col('selftext') != '[removed]') &
    (pl.col('selftext').str.n_chars() > 10)
)

In [33]:
p_df.collect()

id,url,num_comments,subreddit,title,selftext,score
str,str,i64,str,str,str,i64
"""6lyur0""","""https://www.re…",0,"""monerosupport""","""Monero Bounty …","""The links hack…",1
"""6owsgj""","""https://www.re…",2,"""monerosupport""","""[QUESTION] Can…","""I transferred …",1
"""6wfbf4""","""https://www.re…",3,"""monerosupport""","""payment comple…","""Hi, sorry for …",2
"""6wk0hq""","""https://www.re…",5,"""monerosupport""","""[QUESTION] Str…","""I generated a …",2
"""6wr31k""","""https://www.re…",3,"""monerosupport""","""Monero GUI - E…","""Searching arou…",3
"""6y36ie""","""https://www.re…",1,"""monerosupport""","""Never recieved…","""I used changer…",1
"""6ymb9v""","""https://www.re…",0,"""monerosupport""","""re-download bl…","""So i've just g…",3
"""6yuh0l""","""https://www.re…",2,"""monerosupport""","""How do I expor…","""I'm having tro…",3
"""6yvlpz""","""https://www.re…",4,"""monerosupport""","""Running a node…","""Hi, I`m planni…",5


In [43]:
c_df = pl.scan_ndjson(
    comments_file
).select(
    pl.col(['parent_id', 'link_id', 'body', 'score', 'id', 'subreddit'])
).filter((
    (pl.col('parent_id') == pl.col('link_id')) &  # maybe need to confirm....
    (pl.col('body') != '[deleted]') &
    (pl.col('body') != '[removed]') &
    (pl.col('score') > 1) 
)).sort(
    ['link_id', 'score'], 
    descending=True
).groupby("link_id", maintain_order=True).agg([
    pl.col('subreddit').first(),
    pl.col('body').head(3),
    pl.col('score').head(3),
    pl.col('id').head(3),
    pl.col('id').count().alias('comment_count'),    
]).with_columns(
    pl.col('link_id').str.slice(3).alias('link_id')
)

In [44]:
c_df.collect()

link_id,subreddit,body,score,id,comment_count
str,str,list[str],list[i64],list[str],u32
"""zzqd0g""","""monerosupport""","[""When did you start synchronizing and how long did you expected it to take? Do you mean wallet synching or blockchain synching? ETA: Because both the daemon and wallet are contained in the Wallet GUI, it would be helpful to know if you truly are talking about the wallet synching.""]",[2],"[""j2dhhyn""]",1
"""zyj90p""","""monerosupport""","[""It depends if you are going to spend part of the same outputs. Note that tracking transactions is still hard even though you would use the same outputs you just received but it still makes it easier to try because the outputs are so closely together so ring signature pool wouldn't be that large if I have understood it correctly. Waiting an hour after getting your transactions is a good way to reduce traceability but you still can spend other outputs during that period without it endangering your privacy. Youtube is a good source to look for some explanations about Monero as well as the project's own site.""]",[2],"[""j26cwj9""]",1
"""zwqn10""","""monerosupport""","[""Does it allow you to edit and control your firewall?""]",[2],"[""j1xr3b5""]",1
"""zvxt3p""","""monerosupport""","[""IRS and other government organisations know what is happenning on the CEX like binance and others. IRS will know that you've bought and withdrawn XMR and how much. If you want to safely store your XMR without them knowing when you've used them from cold storage, you better buy XMR and then use temporary wallet in between CEX and cold storage, so CEX wouldn't know your wallet address. If you want gov to not know if you have XMR and how much, you'd either need to buy XMR using P2P platforms or buy BTC or other coin(LTC for example) and exchange it for XMR on some DEX. There are services, where you can buy prepaid debit cards with XMR."", ""Or....(and I'm NOT telling you to do something illegal) Buy USDC from a CEX then send it to a DEX. Once your USDC is on a DEX exchange it to XMR. Send your XMR to a cold storage. Also, look at HAVEN (XHV) and you may be able to do something through the Haven protocol. My thought is if you obtain USDC with fiat, then send the USDC to a private wallet, it's got to be more difficult for the IRS to track it. Also, IMO, buying USDC is not buying what the IRS may refer to as a security (taxable). If that's too much (yes, you may have to pay some fees for sending your crypto to more than one location) then just explain to the IRS that you sent your USDC to ""somewhere"" you thought was the correct address...but it wasn't so you lost it forever in the ether world! (That, or a boating accident!) \*\*\*Truth be told, these crypto laws are so vague that even my H&amp;R Block CPA wasn't sure how, exactly, certain cryptos should be filed.""]","[3, 2]","[""j1sjpex"", ""j1uikzg""]",2
"""zvt9rl""","""monerosupport""","[""You're doing something weird for it to take that long on fibet with SSD but to help I would need more information about what wallet you use, if you're running a local or remote node, etc. Can you explain exactly what you've been doing? And maybe what your goal is?""]",[3],"[""j1ri29d""]",1
"""zv29j1""","""monerosupport""","[""Did you use the Ledger Monero seed conversion tool by LedgerHQ? The 25 word seed is for Monero wallets. The original Ledger 24 word seed is for Ledger devices only but the device will generate your original Monero wallet from that seed, not from the 25 word seed.""]",[2],"[""j1mnsl7""]",1
"""zubb0v""","""monerosupport""","[""What do you mean? If you can see that text, you have your GUI wallet open""]",[2],"[""j1i60go""]",1
"""zu6stg""","""monerosupport""","[""At what step(s) did you actually lose money? Which exchanges are you using? Are you okay with KYC (generally lower fees)? Centralized Exchanges generally have the lostest fees on trading. Generally less than 0.5%. Then withdrawal fees is another thing to consider. A good exchanges

In [51]:
j_df = p_df.join(
    c_df, 
    left_on='id', 
    right_on='link_id', 
    how='inner', 
    suffix='_comment'
).explode('body', 'score_comment', 'id_comment')

In [55]:
o_df = j_df.collect()

In [56]:
with open('data/training_ds', mode="ab") as f:
    o_df.write_ndjson(f)